# In this version, I'm trying to see if I can confirm if an employee is present only if he is showing in the frames (video feed) for about 3 seconds to make it more robust.

https://www.youtube.com/watch?v=IT53xBR1A7M&list=PLqUHmcsDDjLjFSo4iumsXGhQoaNu-aXUF

### This jupyter notebook is used to detect faces in images (using MTCNN) stored in `photos` directory and extract their features (using InceptionResnetV1) and store them in `data.pt` file. This file is then loaded later in order to extract the features of faces stored in it and compare it with the face(s) detected in each live frame from a camera stream. If there is a match according to some threshold, then it will label the person in the frame.
***

In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image
import cv2
import time
import os

In [2]:
# Initializing MTCNN and InceptionResnetv1

mtcnn0 = MTCNN(image_size=240, margin=0, keep_all=False, min_face_size=40) # keep_all = False means if one img contains many faces then it will keep only 1 face of those
mtcnn = MTCNN(image_size=240, margin=0, keep_all=True, min_face_size=300) # keep_all = True keeps all faces in a form of list, Also changed mmin_face_size to 150 to ignore small faces in frame which might be difficult to classify
resnet = InceptionResnetV1(pretrained='vggface2').eval() # creating an instance of pre-trained InceptionResnetV1 model trained on the VGGFace2 dataset to extract embeddings

In [3]:
# Read data from directory

dataset = datasets.ImageFolder('photos') # photos directory path
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # new dictionary of indecis and class names
idx_to_class

{0: 'Loei_Haidari', 1: 'Najwa', 2: 'Taher_Haidari', 3: 'Yousef_Haidari'}

In [4]:
def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)
loader

In [5]:
name_list = [] # list of names corresponding to cropped photos
embedding_list = [] # list of embedding matrix after conversion from cropped faces to embedding matrix using resnet

In [6]:
for img, idx in loader: # img is in PIL format, here we are looping thru the images in the photos directory (loader)
    face, prob = mtcnn0(img, return_prob=True) # img passed into mtcnn0, face is the cropped face (note: in mtcnn0 it will return only ONE face)
    if face is not None and prob>0.92: # if face is not None means that there's a face that exists 
        emb = resnet(face.unsqueeze(0)) # pass the face into resnet, unsqueeze b/c resnet expects 4 dimensions (dimension of batch included). Result of this is an embedding
        embedding_list.append(emb.detach()) # .detach() to make requires_grad false
        name_list.append(idx_to_class[idx])

# save data
data = [embedding_list, name_list] # make a new list of the previous 2 lists
torch.save(data, 'data.pt') # saving to make this code reusable without having to go thru the previous loops and steps every single time which is very costly

# Using webcam to recognize the face

In [11]:
import pandas as pd
import time

# Loading data.pt file
load_data = torch.load('data.pt')
embedding_list = load_data[0] 
name_list = load_data[1]

cam = cv2.VideoCapture(0) # initializing the camera from CV, 0 means the default webcam in the device

names = [name.replace('_', ' ') for name in name_list]
attendance_dict = {name: {"status": "absent", "arrival_date": "", "arrival_time": "", "departure_date": "", "departure_time": ""} for name in names} # Initialize attendance dictionary which has all the names in the db as keys and another dictionary as value

# Initialize counter and maximum consecutive frames
consecutive_frames_dict = {name: 0 for name in names} # all names in db as keys and 0 for their values
max_consecutive_frames = 50 # increase/decrease this to modify time needed to take attendance

# Convert the attendance_dict to a Pandas DataFrame
df_attendance = pd.DataFrame.from_dict(attendance_dict, orient='index')

# Reset the index to get the names as a column instead of an index
df_attendance.reset_index(inplace=True)
df_attendance.rename(columns={'index': 'Name'}, inplace=True)

# Print the DataFrame
print("\nAttendance information before program starts:")
print(df_attendance)

# Initialize a dictionary to keep track of the last frame number where an employee's status was changed to "present"
last_present_frame_dict = {name: -1 for name in names}

while True: # read all the frames in the video
    ret, frame = cam.read() # return is true or false, if the webcam succesfully captures an image then it is true otherwise false. frame reperesnts a single frame (image) captured from the webcam using VideoCapture object
    if not ret:
        print("\nFail to grab frame, try again")
        break
    
    img = Image.fromarray(frame) # Image class is part of PIL, .fromarray() is a method that creates an "Image" object from a numpy array 
    img_cropped_list, prob_list = mtcnn(img, return_prob=True) # pass the image into mtcnn, mtcnn will return multiple faces if the image contain multiple faces, also return all the probabilities for all the faces

    face_detected: bool

    if img_cropped_list is not None:
        # If there's a face, make face_detected = true and increment the counter of that identity
        face_detected = True
        try:
            consecutive_frames_dict[name] += 1
        except:
            # Handle the NameError here
            print("NameError occured")
            pass
    
    elif img_cropped_list is None:
        # If there's no face, make face_detected = false and re-initialize the counter back to 0. This is done to make sure that a face is detected in CONSECUTIVE frames if not, the counter is back to 0
        face_detected = False
        try:
            consecutive_frames_dict[name] = 0
        except NameError:
            # Handle the NameError here
            print("NameError occured")
            pass

    if img_cropped_list is not None: # if the image has at least one face
        boxes, _ = mtcnn.detect(img) # return the boxes of faces 
        # print(boxes[0][0])

        for i, prob in enumerate(prob_list): # loop thru the prob list
            if prob > 0.90: 
                emb = resnet(img_cropped_list[i].unsqueeze(0)).detach()
                
                dist_list = [] # for the distance between the current embedding and embeddings of faces in the photos directory (similarity). Minimum distance is used to identify the person

                for idx, emd_db in enumerate(embedding_list):
                    dist = torch.dist(emb, emd_db).item() # calc. distance betweem current embedding and embeddings stored embedding_list
                    dist_list.append(dist)

                # print(f"Minimum distance array:\n{dist_list}")
                min_dist = min(dist_list) # get minimum dist value
                # formatted_min_dist = f'{min_dist:.4f}' # used for printing on the frame
                min_dist_idx = dist_list.index(min_dist) # get minimum dist index (where the min dist is located in the array)
                name = names[min_dist_idx] # get name corresponding to minium dist ##########
                name = name.replace('_', ' ')

                box = boxes[i]
 
                if name in consecutive_frames_dict: # when a face is detected, we retrieve the recognized name
                    if consecutive_frames_dict[name] >= max_consecutive_frames: 
                        # If the counter reaches or exceeds the max_consecutive_frames, we update the attendance status of the person in the attendance_dict from "absent" to "present"
                        if attendance_dict[name]["status"] == "absent":
                            
                            current_date = time.strftime("%Y-%m-%d")
                            current_time = time.strftime("%H:%M:%S")

                            print(f"\n[INFO] {name} arrived at: {current_time}")

                            # Update the date and specific time of face detection
                            attendance_dict[name]["arrival_time"] = current_time
                            attendance_dict[name]["arrival_date"] = current_date

                            # Update status
                            attendance_dict[name]["status"] = "present"

                            # Update the last_present_frame_dict to keep track of the frame number
                            last_present_frame_dict[name] = consecutive_frames_dict[name]

                            # Update the DataFrame with the latest attendance information
                            df_attendance["status"] = df_attendance["Name"].map(lambda name: attendance_dict[name]["status"])
                            df_attendance["arrival_date"] = df_attendance["Name"].map(lambda name: attendance_dict[name]["arrival_date"])
                            df_attendance["arrival_time"] = df_attendance["Name"].map(lambda name: attendance_dict[name]["arrival_time"])

                        elif consecutive_frames_dict[name] - last_present_frame_dict[name] >= (max_consecutive_frames):
                            # If the current frame number minus the last_present_frame is greater than or equal to max_consecutive_frames,
                            # we change the status back to "absent" after a certain number of frames have passed since the employee's last presence.
                            if attendance_dict[name]["status"] == "present":

                                current_date = time.strftime("%Y-%m-%d")
                                current_time = time.strftime("%H:%M:%S")

                                print(f"\n[INFO] {name} departed at: {current_time}")

                                # Update the date and specific time of face detection
                                attendance_dict[name]["departure_time"] = current_time
                                attendance_dict[name]["departure_date"] = current_date

                                attendance_dict[name]["status"] = "departed"

                                # Update the DataFrame with the latest attendance information
                                df_attendance["status"] = df_attendance["Name"].map(lambda name: attendance_dict[name]["status"])
                                df_attendance["departure_date"] = df_attendance["Name"].map(lambda name: attendance_dict[name]["departure_date"])
                                df_attendance["departure_time"] = df_attendance["Name"].map(lambda name: attendance_dict[name]["departure_time"])
                else:
                    consecutive_frames_dict[name] = 0

                original_frame = frame.copy() # storing a copy of frame before drawing on it
        
                if min_dist<0.90:
                    # frame = cv2.putText(frame, name+' '+str(formatted_min_dist), (int(box[0])-5, int(box[1])-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
                    # Display employee name and status on the frame
                    if attendance_dict[name]["status"] == "present":
                        status_color = (0, 255, 0)
                        status_text = "Arrived"
                        frame = cv2.putText(frame, f"{name} - {status_text}", (int(box[0])-5, int(box[1])-12), cv2.FONT_HERSHEY_SIMPLEX, 1, status_color, 2, cv2.LINE_AA)
                        frame = cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), status_color, 3)

                    elif attendance_dict[name]["status"] == "departed":
                        status_color = (255, 0, 0)
                        status_text = "Departed"
                        frame = cv2.putText(frame, f"{name} - {status_text}", (int(box[0])-5, int(box[1])-12), cv2.FONT_HERSHEY_SIMPLEX, 1, status_color, 2, cv2.LINE_AA)
                        frame = cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), status_color, 3)

                    else:
                        status_color = (0, 0, 255)
                        status_text = "Absent"
                        frame = cv2.putText(frame, f"{name} - {status_text}", (int(box[0])-5, int(box[1])-12), cv2.FONT_HERSHEY_SIMPLEX, 1, status_color, 2, cv2.LINE_AA)
                        frame = cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), status_color, 3)

    cv2.imshow("IMG", frame)

    k = cv2.waitKey(1)
    # Press q or esc to quit, space bar to add & save new image
    if k == ord('q'):
        break # Quit the program

    elif k == 27: # 27 is for esc key
        break

    elif k == 32: # 32 is the ASCII of space bar
        print('Enter your name: ')
        name = input()
        if name == "": ###### I ADDED THESE 2 LINES IN CASE INPUT WAS EMPTY IT DOESN'T SAVE TEH IMAGE
            continue

        # Create directory if class/person does not exist in photos directory
        if not os.path.exists('photos/'+name):
            os.mkdir('photos/'+name) 

        img_name = f"photos/{name}/{int(time.time())}.jpg"
        cv2.imwrite(img_name, original_frame)
        print(f"saved: {img_name}")

cam.release()

# Print the DataFrame
print("\nAttendance information after program stopped:")
print(df_attendance)

# print()
# print(attendance_dict)

# # Export the DataFrame to a CSV or xlsx (excel) file
# excel_file = 'attendance.xlsx'
# # Use  .to_excel() or .to_csv() method to export dataframe
# df_attendance.to_excel(excel_file, index=False) # index=False argument ensures that the row indices are not included in the CSV file.
# print(f"\n[INFO] Attendance data has been exported to '{excel_file}' successfully.")

for i in range(1):
    cv2.destroyAllWindows()
    cv2.waitKey(1)

# OR the below method, but below method requires one moqre key press to destroy the window while the above loop automatically closes the window without extra key press

# cv2.waitKey(0)
# cv2.destroyAllWindows()
# cv2.waitKey(1) # This is the only extra line u need on top of windows os uses to work properly on mac


Attendance information before program starts:
             Name  status arrival_date arrival_time departure_date  \
0    Loei Haidari  absent                                            
1           Najwa  absent                                            
2   Taher Haidari  absent                                            
3  Yousef Haidari  absent                                            

  departure_time  
0                 
1                 
2                 
3                 

[INFO] Yousef Haidari arrived at: 20:57:50

Attendance information after program stopped:
             Name   status arrival_date arrival_time departure_date  \
0    Loei Haidari   absent                                            
1           Najwa   absent                                            
2   Taher Haidari   absent                                            
3  Yousef Haidari  present   2023-07-23     20:57:50                  

  departure_time  
0                 
1                 
2  

In [8]:
# !jupyter nbconvert --to script face_reco.ipynb